In [ ]:
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")

In [ ]:
#parameters for the source folder and the json list of child tables to loop through
#this is passed in by a Data Pipeline
source_folder = ''
source_table_list = ''


In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import ast
import json

###########################################################
#create function to save each source folder to a Delta table
def loadDataFromSource(source_folder, table_name):

    #load raw data from Files
    df = spark.read.format("parquet").load('Files/' + source_folder + '/' + table_name)

    df = df.withColumn("RawLoadDateTime", current_timestamp()).withColumn("RawSourceFolder", lit(source_folder))

    #append new metadata columns and save to Delta table
    df.write.mode("append").format("delta").option("mergeSchema", "true").save("Tables/" + table_name)
###########################################################

#load json list of tables
table_list = json.loads(source_table_list)

#create new empty list
table_result_list = []

#loop over json object and save name of table to list
for json_dict in table_list:
    table_result_list.append(json_dict['name'])

for i in table_result_list:
    loadDataFromSource(source_folder,i)